In [24]:
import numpy as np
import pandas as pd
#import cPickle
from collections import defaultdict
import re

from bs4 import BeautifulSoup

import sys
import os


from keras.preprocessing.text import Tokenizer, text_to_word_sequence
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical

from keras.layers import Embedding
from keras.layers import Dense, Input, Flatten
from keras.layers import Conv1D, MaxPooling1D, Embedding, Dropout, LSTM, GRU, Bidirectional, TimeDistributed, SpatialDropout1D
from keras.models import Model
from keras.layers.normalization import BatchNormalization

from keras import backend as K
from keras.engine.topology import Layer, InputSpec
from keras import initializers, regularizers, constraints


In [35]:
MAX_NB_WORDS = 80000
MAX_SEQUENCE_LENGTH = 400
VALIDATION_SPLIT = 0.2
EMBEDDING_DIM = 100

In [3]:
data_train = pd.read_csv('/Users/venkatakrishnamohansunkara/Desktop/DM/classification_data.csv', sep=',')
print(data_train.shape)
data_train.dropna(inplace=True)

(17575, 7)


In [4]:
data_train['cat_id'] = 0
for i,row in data_train.iterrows():
    if row['Category'] == 'unrest':
        data_train.set_value(i,'cat_id',0)
    else:
        data_train.set_value(i,'cat_id',1)

/anaconda3/envs/textClassifier/lib/python3.6/site-packages/ipykernel_launcher.py:4: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  after removing the cwd from sys.path.
/anaconda3/envs/textClassifier/lib/python3.6/site-packages/ipykernel_launcher.py:6: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  


In [5]:
X_data = data_train['Content']
y_data = data_train['cat_id']

In [6]:
X_data = list(X_data)
y_data = list(y_data)

In [7]:
tokenizer = Tokenizer(num_words=MAX_NB_WORDS,oov_token=1)
tokenizer.fit_on_texts(X_data)
X_data = tokenizer.texts_to_sequences(X_data)

In [17]:
#X_data = str(X_data)
X_data = pad_sequences(X_data,
        maxlen=MAX_SEQUENCE_LENGTH,
        padding='post',
        truncating='post',
        dtype='float32')

In [20]:
y_data = np.asarray(y_data)

In [21]:
y_data.shape

(17467,)

In [54]:
y_data = to_categorical(np.asarray(y_data))

In [55]:
indices = np.arange(X_data.shape[0])
np.random.shuffle(indices)
X_data = X_data[indices]
y_data = y_data[indices]
nb_validation_samples = int(VALIDATION_SPLIT * X_data.shape[0])

x_train = X_data[:-nb_validation_samples]
y_train = y_data[:-nb_validation_samples]
x_val = X_data[-nb_validation_samples:]
y_val = y_data[-nb_validation_samples:]

print('Number of positive and negative reviews in traing and validation set')
print(y_train.sum(axis=0))
print(y_val.sum(axis=0))

Number of positive and negative reviews in traing and validation set
[6869. 7105.]
[1706. 1787.]


In [33]:
GLOVE_DIR = "/Users/venkatakrishnamohansunkara/Desktop/DM/textClassifier"
embeddings_index = {}
f = open(os.path.join(GLOVE_DIR, 'glove.6B.100d.txt'))
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Total %s word vectors.' % len(embeddings_index))

Total 400000 word vectors.


In [40]:
word_index = tokenizer.word_index
# Embedding matrix should consist of all the unique words (which is the lenght of word index) as rows and their embeddings as columns (100 dimensions)
embedding_matrix = np.random.random((len(word_index) + 1, EMBEDDING_DIM))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
         #words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector
    else:
        embedding_matrix[i] = 100*[0]

In [41]:
def dot_product(x, kernel):
    """
    Wrapper for dot product operation, in order to be compatible with both
    Theano and Tensorflow
    Args:
        x (): input
        kernel (): weights
    Returns:
    """
    if K.backend() == 'tensorflow':
        return K.squeeze(K.dot(x, K.expand_dims(kernel)), axis=-1)
    else:
        return K.dot(x, kernel)
    

class AttentionWithContext(Layer):
    """
    Attention operation, with a context/query vector, for temporal data.
    Supports Masking.
    Follows the work of Yang et al. [https://www.cs.cmu.edu/~diyiy/docs/naacl16.pdf]
    "Hierarchical Attention Networks for Document Classification"
    by using a context vector to assist the attention
    # Input shape
        3D tensor with shape: `(samples, steps, features)`.
    # Output shape
        2D tensor with shape: `(samples, features)`.
    How to use:
    Just put it on top of an RNN Layer (GRU/LSTM/SimpleRNN) with return_sequences=True.
    The dimensions are inferred based on the output shape of the RNN.
    Note: The layer has been tested with Keras 2.0.6
    Example:
        model.add(LSTM(64, return_sequences=True))
        model.add(AttentionWithContext())
        # next add a Dense layer (for classification/regression) or whatever...
    """

    def __init__(self,
                 W_regularizer=None, u_regularizer=None, b_regularizer=None,
                 W_constraint=None, u_constraint=None, b_constraint=None,
                 bias=True, **kwargs):

        self.supports_masking = True
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.u_regularizer = regularizers.get(u_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.u_constraint = constraints.get(u_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        super(AttentionWithContext, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3

        self.W = self.add_weight((input_shape[-1], input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        if self.bias:
            self.b = self.add_weight((input_shape[-1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)

        self.u = self.add_weight((input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_u'.format(self.name),
                                 regularizer=self.u_regularizer,
                                 constraint=self.u_constraint)

        super(AttentionWithContext, self).build(input_shape)

    def compute_mask(self, input, input_mask=None):
        # do not pass the mask to the next layers
        return None

    def call(self, x, mask=None):
        uit = dot_product(x, self.W)

        if self.bias:
            uit += self.b

        uit = K.tanh(uit)
        ait = dot_product(uit, self.u)

        #a = K.exp(ait)
        a = K.softmax(ait)
        # apply mask after the exp. will be re-normalized next
        #if mask is not None:
        #    # Cast the mask to floatX to avoid float64 upcasting in theano
        #    a *= K.cast(mask, K.floatx())

        # in some cases especially in the early stages of training the sum may be almost zero
        # and this results in NaN's. A workaround is to add a very small positive number ε to the sum.
        # a /= K.cast(K.sum(a, axis=1, keepdims=True), K.floatx())
        #a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())
        a = K.expand_dims(a)
        weighted_input = x * a
        return K.sum(weighted_input, axis=1)

    def compute_output_shape(self, input_shape):
        return input_shape[0], input_shape[-1]

In [50]:
input_layer = Input(shape=(MAX_SEQUENCE_LENGTH,),dtype='int32')
embedding_layer = Embedding(input_dim=len(word_index)+1,
        output_dim=EMBEDDING_DIM,
        input_length=MAX_SEQUENCE_LENGTH,
        weights=[embedding_matrix],
        embeddings_regularizer=regularizers.l2(0.00),
        trainable=True)(input_layer)
drop1 = SpatialDropout1D(0.3)(embedding_layer)
lstm_1 = Bidirectional(LSTM(128, name='blstm_1',
    activation='tanh',
    recurrent_activation='hard_sigmoid',
    recurrent_dropout=0.0,
    dropout=0.5, 
    kernel_initializer='glorot_uniform',
    return_sequences=True),
    merge_mode='concat')(drop1)
lstm_1 = BatchNormalization()(lstm_1)
att_layer = AttentionWithContext()(lstm_1)
drop3 = Dropout(0.5)(att_layer)
predictions = Dense(2, activation='sigmoid')(drop3)
model = Model(inputs=input_layer, outputs=predictions)

In [51]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         (None, 400)               0         
_________________________________________________________________
embedding_3 (Embedding)      (None, 400, 100)          10835400  
_________________________________________________________________
spatial_dropout1d_2 (Spatial (None, 400, 100)          0         
_________________________________________________________________
bidirectional_2 (Bidirection (None, 400, 256)          234496    
_________________________________________________________________
batch_normalization_2 (Batch (None, 400, 256)          1024      
_________________________________________________________________
attention_with_context_2 (At (None, 256)               66048     
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0         
__________

In [52]:
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])

In [56]:
print("model fitting - Bi-LSTM attention network")
model.fit(x_train, y_train, validation_data=(x_val, y_val),
          nb_epoch=10, batch_size=50)

model fitting - Hierachical attention network


/anaconda3/envs/textClassifier/lib/python3.6/site-packages/ipykernel_launcher.py:3: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  This is separate from the ipykernel package so we can avoid doing imports until


Train on 13974 samples, validate on 3493 samples
Epoch 1/10
  100/13974 [..............................] - ETA: 1:30:16 - loss: 0.7180 - acc: 0.5500

KeyboardInterrupt: 